In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install ragas langchain_community langchain_groq sentence_transformers xmltodict -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.4/362.4 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os
from google.colab import userdata
import pandas as pd
from langchain_community.document_loaders import PubMedLoader
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_groq import ChatGroq
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

In [4]:
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
data_generation_model = ChatGroq(
    model="llama3-8b-8192",
    groq_api_key=os.environ["GROQ_API_KEY"],
)

In [5]:
critic_model = ChatGroq(
    model="llama3-8b-8192",
    groq_api_key=os.environ["GROQ_API_KEY"],
)

In [6]:
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
loader = PubMedLoader("cancer",load_max_docs=5)

In [8]:
documents = loader.load()

Too Many Requests, waiting for 0.20 seconds...
Too Many Requests, waiting for 0.40 seconds...


In [9]:
generator = TestsetGenerator.from_langchain(
    data_generation_model,
    critic_model,
    embeddings
)

In [10]:
distributions = {
    simple : 0.5,
    multi_context : 0.4,
    reasoning : 0.1

}

In [11]:
testset = generator.generate_with_langchain_docs(documents,5,distributions)

embedding nodes:   0%|          | 0/10 [00:00<?, ?it/s]

Generating:   0%|          | 0/5 [00:00<?, ?it/s]

In [12]:
test_df = testset.to_pandas()

In [13]:
test_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,Here is a question that can be fully answered ...,[The advent of robotic bronchoscopy coupled wi...,The main limitation in the diagnostic yields o...,simple,"[{'uid': '39151149', 'Title': 'The Importance ...",True
1,Here is a question that can be fully answered ...,[BACKGROUND: Physical activity engagement foll...,The domains of the Theoretical Domains Framewo...,simple,"[{'uid': '39151159', 'Title': 'A Theory and Ev...",True
2,Here is a question that can be fully answered ...,[BACKGROUND: Venous thromboembolism is the lea...,The Khorana score has been widely validated in...,simple,"[{'uid': '39151210', 'Title': 'External valida...",True
